In [1]:
import datetime
import itertools
import math
import os
from typing import List, Tuple, Dict, Literal

import joblib
import networkx as nx
import numpy as np
import pandas as pd
  
Papers = List[Tuple[List[int], datetime.datetime]]
Edge = List[Tuple[int, int, Dict[Literal['date'], datetime.datetime]]]

def weighted_median(d: np.ndarray) -> float: return np.average(d[:,0], weights=d[:,1])

# Condmat

In [7]:
def get_edgelist(*, filepath='src/cond-mat.hg2', start=None, stop=None) -> List[Edge]:
  """Return E_[t_1, t_2]."""
  papers = _get_papers(filepath)
  edges = [
    (u, v, dict(date=date)) if u<v else (v, u, dict(date=date))
    for authors, date in papers
    for u, v in itertools.combinations(authors, 2)
  ]
  return _filter_edgelist(edges, start, stop)
def _filter_edgelist(edges: List[Edge], start, stop) -> List[Edge]: 
  """Filter edgelist.  If start/ stop is float, start/stop from the fraction of total edges. If datetime, this is used.""" 
  no_edges = len(edges)
  if start is None: start=0
  if stop is None: stop=1
  if type(start) is float or start == 0:
    start_index = int(start*no_edges)
    start = edges[start_index][2]['date']
  if type(stop) is float or stop == 1:
    stop_index = math.floor(stop*no_edges)-1
    stop = edges[stop_index][2]['date']
  return [edge for edge in edges if edge[2]['date'] >= start and edge[2]['date'] <= stop]
def _get_papers(filepath: str) -> Papers:
  """Read collaboration data in filepath and return all papers."""
  
  papers = list()
  # Get number of rows to read for the vertices.
  with open(filepath) as file:
    no_rows = int(file.readline().split(' ')[1])
 
  with open(filepath) as file:
    for paper in file.readlines()[no_rows+2:]:
      # Each line has the following format: epoch no_authors [ u v (w ...) ]
      epoch = datetime.datetime.fromtimestamp(int(paper.split(' ')[0]))
          
      no_authors = int(paper.split(' ')[1])
      index1 = paper.find('[')+2
      index2 = paper.find(']')-1

      authors = [int(auth) for auth in paper[index1:index2].split(' ')]
      assert no_authors == len(authors)
      
      papers.append((authors, epoch))
  return papers
def get_graph(edgelist: List[Edge]) -> nx.Graph:
  """Add edge to graph. Contains edge attribute weight."""
  g = nx.Graph()
  
  for u, v, _ in edgelist:
    weight = g[u][v]["weight"]+1 if g.has_edge(u,v) else 1
    g.add_edge(u, v, weight=weight)
  
  return g
def giant_component(graph: nx.Graph) -> nx.Graph: return graph.subgraph(max(nx.connected_components(graph), key=len)).copy()

In [17]:
%%time

filepath = 'datasets/condmat/cond-mat.hg2'
print(f'Size of file: {os.path.getsize(filepath):.0e}')
edges = get_edgelist(filepath=filepath)
g = get_graph(edges)

with open()
!printf '%s\n' 'load_undirected temp/network.edges' 'dist_distri' > "temp/input.txt"
nx.write_edgelist(g, 'temp/network.edges', data=False)
! /local/bruingjde/teexgraph/teexgraph < temp/input.txt > temp/output.txt

gc = giant_component(g)
e = g.number_of_edges()
n = g.number_of_nodes()
print(f'Number of edges (gc): {e} ({gc.number_of_edges() / e:.0%})')
print(f'Number of edges (gc): {n} ({gc.number_of_nodes() / n:.0%})')
print(f'Density: {nx.density(g):.1e}')
print(f"Mean distance: {weighted_median(np.loadtxt('temp/output.txt')):.1f}")
print(f"Diameter: {int(np.loadtxt('temp/output.txt')[-1,0])}")

Size of file: 1e+06
*** Welcome to teexGraph ***
- Use standard input (cin) to give commands
- Read standard output (cout) to catch the result
- Observe standard log (clog) and (cerr) for status and error messages
- Graphs up to MAXN = 10000000 nodes are accepted
Input a command: Loading an undirected graph. Enter filename: 
Loading graph from temp/network.edges ...
- 55276 edges added (m = 55276) in total
- 0 edges skipped
- 6 self-edges added

Sorting edge list...
Sorting done.
Loading done.

Making graph undirected (m = 55276)...
Sorting edge list...
Sorting done.
  Verify that the graph is actually undirected.
Undirected-making done (m = 110546).
Loading file succeeded.
WCC computed.

> Computing distance distribution (based on a 100% sample of 17218 nodes) with 128 CPUs...
            0          000   0   0 0   0% 0% 0% 0% 0% 0% 0 %  0% 0%0% 0% 0% 0% 0%0     000%0   0%0% 0% 0% 0% 0% 0% 0% 0    0% 0% 0% 0% 0% 0% 0% 0%0% 0% 0% 0% 0% 0% 0%0%  0%0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0%0% 0

# Enron

In [21]:
def read_edges(file: str, sep=' ', skiprows=1) -> pd.DataFrame:
  d = pd.read_csv(file, sep, skiprows=skiprows, names=['source', 'target', 'weight', 'date'])
  d['date'] = d['date'].apply(datetime.datetime.fromtimestamp)
  d.sort_values(by='date', inplace=True)
  return d.loc[:, ['source', 'target', 'date']]
def get_graph(edgelist: pd.DataFrame) -> nx.Graph:
  """Add edge to graph. Contains edge attribute weight."""
  g = nx.Graph()
  
  for u, v, _ in edgelist.itertuples(index=False, name=None):
    weight = g[u][v]["weight"]+1 if g.has_edge(u,v) else 1
    g.add_edge(u, v, weight=weight)
  
  return g

In [62]:
g = get_graph(read_edges('datasets/enron/out.enron'))

In [64]:
%%time
%%capture
!printf '%s\n' 'load_undirected temp/network.edges' 'dist_distri' > "temp/input.txt"
nx.write_edgelist(g, 'temp/network.edges', data=False)
! /local/bruingjde/teexgraph/teexgraph < temp/input.txt > temp/output.txt

CPU times: user 543 ms, sys: 68.8 ms, total: 612 ms
Wall time: 25.5 s


In [65]:
gc = giant_component(g)
e = g.number_of_edges()
n = g.number_of_nodes()
print(f'Number of edges (gc): {e} ({gc.number_of_edges() / e:.0%})')
print(f'Number of edges (gc): {n} ({gc.number_of_nodes() / n:.0%})')
print(f'Density: {nx.density(g):.1e}')
print(f"Mean distance: {weighted_median(np.loadtxt('temp/output.txt')):.1f}")
print(f"Diameter: {int(np.loadtxt('temp/output.txt')[-1,0])}")

Number of edges (gc): 299220 (99%)
Number of edges (gc): 87273 (97%)
Density: 7.9e-05
Mean distance: 4.9
Diameter: 14


# Askubuntu

In [72]:
g = get_graph(read_edges('datasets/askubuntu/out.sx-askubuntu', sep='\t'))

In [73]:
%%time
%%capture
!printf '%s\n' 'load_undirected temp/network.edges' 'dist_distri' > "temp/input.txt"
nx.write_edgelist(g, 'temp/network.edges', data=False)
! /local/bruingjde/teexgraph/teexgraph < temp/input.txt > temp/output.txt

CPU times: user 1.26 s, sys: 193 ms, total: 1.45 s
Wall time: 2min 3s


In [74]:
gc = giant_component(g)
e = g.number_of_edges()
n = g.number_of_nodes()
print(f'Number of edges (gc): {e} ({gc.number_of_edges() / e:.0%})')
print(f'Number of edges (gc): {n} ({gc.number_of_nodes() / n:.0%})')
print(f'Density: {nx.density(g):.1e}')
print(f"Mean distance: {weighted_median(np.loadtxt('temp/output.txt')):.1f}")
print(f"Diameter: {int(np.loadtxt('temp/output.txt')[-1,0])}")

Number of edges (gc): 508003 (99%)
Number of edges (gc): 159316 (96%)
Density: 4.0e-05
Mean distance: 3.9
Diameter: 13


# Bibsonomy

In [75]:
g = get_graph(read_edges('datasets/bibsonomy/out.bibsonomy-2ui'))

In [79]:
assert g.number_of_nodes() < 1e8

In [80]:
%%time
!printf '%s\n' 'load_undirected temp/network.edges' 'dist_distri' > "temp/input.txt"
nx.write_edgelist(g, 'temp/network.edges', data=False)
! /local/bruingjde/teexgraph/teexgraph < temp/input.txt > temp/output.txt

*** Welcome to teexGraph ***
- Use standard input (cin) to give commands
- Read standard output (cout) to catch the result
- Observe standard log (clog) and (cerr) for status and error messages
- Graphs up to MAXN = 10000000 nodes are accepted
Input a command: Loading an undirected graph. Enter filename: 
Loading graph from temp/network.edges ...
- 801782 edges added (m = 801782) in total
- 0 edges skipped
- 3 self-edges added

Sorting edge list...
Sorting done.
Loading done.

Making graph undirected (m = 801782)...
Sorting edge list...
Sorting done.
  Verify that the graph is actually undirected.
Undirected-making done (m = 1603561).
Loading file succeeded.
WCC computed.

> Computing distance distribution (based on a 100% sample of 767447 nodes) with 128 CPUs...
   0     0                       0    0000 0 0 0  %% 0 00 0%% 0   %0 0 %0 0  %0000% % 0   0 0 %0%  00%%   00%0  0%0% 0% 0% 00% 0   0% 0%0% 0%0% 0%0%0% 0%0%0%0% 0% 0%% 0%0%0%0% 0%0% 0% 0% 0% 0% 0%%0%% 0% 0% 0% 0% 0%0%% 0%% 0%%0

In [81]:
gc = giant_component(g)
e = g.number_of_edges()
n = g.number_of_nodes()
print(f'Number of edges (gc): {e} ({gc.number_of_edges() / e:.0%})')
print(f'Number of edges (gc): {n} ({gc.number_of_nodes() / n:.0%})')
print(f'Density: {nx.density(g):.1e}')
print(f"Mean distance: {weighted_median(np.loadtxt('temp/output.txt')):.1f}")
print(f"Diameter: {int(np.loadtxt('temp/output.txt')[-1,0])}")

Number of edges (gc): 801782 (100%)
Number of edges (gc): 767447 (100%)
Density: 2.7e-06
Mean distance: 4.9
Diameter: 6


# Digg

In [82]:
g = get_graph(read_edges('datasets/digg/out.munmun_digg_reply'))

In [84]:
assert g.number_of_nodes() < 1e8
%%time
!printf '%s\n' 'load_undirected temp/network.edges' 'dist_distri' > "temp/input.txt"
nx.write_edgelist(g, 'temp/network.edges', data=False)
! /local/bruingjde/teexgraph/teexgraph < temp/input.txt > temp/output.txt

*** Welcome to teexGraph ***
- Use standard input (cin) to give commands
- Read standard output (cout) to catch the result
- Observe standard log (clog) and (cerr) for status and error messages
- Graphs up to MAXN = 10000000 nodes are accepted
Input a command: Loading an undirected graph. Enter filename: 
Loading graph from temp/network.edges ...
- 86312 edges added (m = 86312) in total
- 0 edges skipped
- 1157 self-edges added

Sorting edge list...
Sorting done.
Loading done.

Making graph undirected (m = 86312)...
Sorting edge list...
Sorting done.
  Verify that the graph is actually undirected.
Undirected-making done (m = 171467).
Loading file succeeded.
WCC computed.

> Computing distance distribution (based on a 100% sample of 30398 nodes) with 128 CPUs...
    0         0  0  0 0% 0% 0%  0  00  0%0   0%   0% 0% 0% 0% 0% 0%0% 0%0% 0 0%  0%0% 0% 0% 0% 0% 0% 0%0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0%  0% 0%0% 0%0% 0%0% 0% 0% 0%0% 0%0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0%% 0%0% 0%0

In [85]:
gc = giant_component(g)
e = g.number_of_edges()
n = g.number_of_nodes()
print(f'Number of edges (gc): {e} ({gc.number_of_edges() / e:.0%})')
print(f'Number of edges (gc): {n} ({gc.number_of_nodes() / n:.0%})')
print(f'Density: {nx.density(g):.1e}')
print(f"Mean distance: {weighted_median(np.loadtxt('temp/output.txt')):.1f}")
print(f"Diameter: {int(np.loadtxt('temp/output.txt')[-1,0])}")

Number of edges (gc): 86312 (100%)
Number of edges (gc): 30398 (98%)
Density: 1.9e-04
Mean distance: 4.7
Diameter: 12


# Edit-enwiki

In [20]:
%%time
g = get_graph(read_edges('datasets/edit-enwiki/out.edit-enwiki', sep='\t'))

CPU times: user 3h 3min 32s, sys: 23min 38s, total: 3h 27min 11s
Wall time: 3h 26min 52s


In [26]:
%%time
joblib.dump(g, 'datasets/edit-enwiki/g.pkl', protocol=5)

CPU times: user 2h 22min 55s, sys: 20min 27s, total: 2h 43min 23s
Wall time: 2h 43min 10s


['datasets/edit-enwiki/g.pkl']

In [ ]:
%%time 
g = joblib.load('datasets/edit-enwiki/g.pkl')

In [ ]:
gc = giant_component(g)
e = g.number_of_edges()
n = g.number_of_nodes()
print(f'Number of edges (gc): {e} ({gc.number_of_edges() / e:.0%})')
print(f'Number of edges (gc): {n} ({gc.number_of_nodes() / n:.0%})')
print(f'Density: {nx.density(g):.1e}')

In [ ]:
%%time
assert g.number_of_nodes() < 1e8
!printf '%s\n' 'load_undirected temp/network.edges' 'dist_distri' > "temp/input.txt"
nx.write_edgelist(g, 'temp/network.edges', data=False)
! /local/bruingjde/teexgraph/teexgraph < temp/input.txt > temp/output.txt
print(f"Mean distance: {weighted_median(np.loadtxt('temp/output.txt')):.1f}")
print(f"Diameter: {int(np.loadtxt('temp/output.txt')[-1,0])}")

# Internet

In [4]:
%%time
g = get_graph(read_edges('datasets/internet/out.topology'))

CPU times: user 714 ms, sys: 36.8 ms, total: 751 ms
Wall time: 750 ms


In [14]:
%%time
assert g.number_of_nodes() < 1e8
!printf '%s\n' 'load_undirected temp/network.edges' 'dist_distri' > "temp/input.txt"
nx.write_edgelist(g, 'temp/network.edges', data=False)
! /tmp/bruingjde/teexgraph/teexgraph < temp/input.txt > temp/output.txt

*** Welcome to teexGraph ***
- Use standard input (cin) to give commands
- Read standard output (cout) to catch the result
- Observe standard log (clog) and (cerr) for status and error messages
- Graphs up to MAXN = 10000000 nodes are accepted
Input a command: Loading an undirected graph. Enter filename: 
Loading graph from temp/network.edges ...
- 107720 edges added (m = 107720) in total
- 0 edges skipped
- 0 self-edges added

Sorting edge list...
Sorting done.
Loading done.

Making graph undirected (m = 107720)...
Sorting edge list...
Sorting done.
Undirected-making done (m = 215440).
Loading file succeeded.
WCC computed.

> Computing distance distribution (based on a 100% sample of 34761 nodes) with 256 CPUs...
    0    0%0          0% 0% 0%  0       0% 0% 0%  0 % 0% 0% 00%% 0% 0%       0      0% 0% 0%   0 %     0% 0% 0% 0% 0% 0%0% 0% 0% 0% 0% 00% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0% 0%0% 0% 0% 0%%0% 0% 0% 0% 0% 0% 0% 0%0%  00% 0% 0%0%0%0% 0%% 0 0%0% 00% 0% 0% 0% 0% 0%

In [9]:
gc = giant_component(g)
e = g.number_of_edges()
n = g.number_of_nodes()
print(f'Number of edges (gc): {e} ({gc.number_of_edges() / e:.0%})')
print(f'Number of edges (gc): {n} ({gc.number_of_nodes() / n:.0%})')
print(f'Density: {nx.density(g):.1e}')

In [15]:
print(f"Mean distance: {weighted_median(np.loadtxt('temp/output.txt')):.1f}")
print(f"Diameter: {int(np.loadtxt('temp/output.txt')[-1,0])}")

Mean distance: 3.8
Diameter: 10


# Slashdot

In [22]:
%%time
g = get_graph(read_edges('datasets/slashdot/out.slashdot-threads', skiprows=2))

CPU times: user 888 ms, sys: 28.1 ms, total: 916 ms
Wall time: 916 ms


In [23]:
%%time
assert g.number_of_nodes() < 1e8
!printf '%s\n' 'load_undirected temp/network.edges' 'dist_distri' > "temp/input.txt"
nx.write_edgelist(g, 'temp/network.edges', data=False)
! /tmp/bruingjde/teexgraph/teexgraph < temp/input.txt > temp/output.txt

*** Welcome to teexGraph ***
- Use standard input (cin) to give commands
- Read standard output (cout) to catch the result
- Observe standard log (clog) and (cerr) for status and error messages
- Graphs up to MAXN = 10000000 nodes are accepted
Input a command: Loading an undirected graph. Enter filename: 
Loading graph from temp/network.edges ...
- 117378 edges added (m = 117378) in total
- 0 edges skipped
- 805 self-edges added

Sorting edge list...
Sorting done.
Loading done.

Making graph undirected (m = 117378)...
Sorting edge list...
Sorting done.
  Verify that the graph is actually undirected.
Undirected-making done (m = 233951).
Loading file succeeded.
WCC computed.

> Computing distance distribution (based on a 100% sample of 51083 nodes) with 256 CPUs...
    00        00% 0%   0 0 %%  0%    0      0     0%      00   00  0%0%   0%0%   % 0%0  0%0%0% 0%%00%0  %  0%  0    0%0%  0%0  0% 0%0  0% 0% 0% 0% 0% 0% 0%0 %0 0%   00% %           00% 0%0 0% 0% 0  0 0%00%  00 %0 %         0% 

In [24]:
gc = giant_component(g)
e = g.number_of_edges()
n = g.number_of_nodes()
print(f'Number of edges (gc): {e} ({gc.number_of_edges() / e:.0%})')
print(f'Number of edges (gc): {n} ({gc.number_of_nodes() / n:.0%})')
print(f'Density: {nx.density(g):.1e}')
print(f"Mean distance: {weighted_median(np.loadtxt('temp/output.txt')):.1f}")
print(f"Diameter: {int(np.loadtxt('temp/output.txt')[-1,0])}")

Number of edges (gc): 117378 (100%)
Number of edges (gc): 51083 (100%)
Density: 9.0e-05
Mean distance: 4.5
Diameter: 17


# Stackexchange

In [26]:
%%time
g = get_graph(read_edges('datasets/stackexchange/out.stackexchange-stackoverflow', skiprows=2))

CPU times: user 10.1 s, sys: 270 ms, total: 10.4 s
Wall time: 10.4 s


In [27]:
%%time
assert g.number_of_nodes() < 1e8
!printf '%s\n' 'load_undirected temp/network.edges' 'dist_distri' > "temp/input.txt"
nx.write_edgelist(g, 'temp/network.edges', data=False)
! /tmp/bruingjde/teexgraph/teexgraph < temp/input.txt > temp/output.txt

*** Welcome to teexGraph ***
- Use standard input (cin) to give commands
- Read standard output (cout) to catch the result
- Observe standard log (clog) and (cerr) for status and error messages
- Graphs up to MAXN = 10000000 nodes are accepted
Input a command: Loading an undirected graph. Enter filename: 
Loading graph from temp/network.edges ...
- 1301887 edges added (m = 1301887) in total
- 0 edges skipped
- 19 self-edges added

Sorting edge list...
Sorting done.
Loading done.

Making graph undirected (m = 1301887)...
Sorting edge list...
Sorting done.
  Verify that the graph is actually undirected.
Undirected-making done (m = 2603755).
Loading file succeeded.
WCC computed.

> Computing distance distribution (based on a 100% sample of 545196 nodes) with 256 CPUs...
 0%     00     0% 0% 0% 0% 0    0 0 0%  0  0      0   0     0 0%0    0%0% 0%  0% 0% 0% 0%  0%0    0 %0% 0% 0%0%% 0%  0%0% 0%% 0% 0%  0        0% 0 00% %0%  0%0% 0% 0% 0%0% 0% 0% 0% 0% 0% 0% 0% 0% 0%0% 0% 0% 0% 0% 0% 0% 0% 

In [28]:
gc = giant_component(g)
e = g.number_of_edges()
n = g.number_of_nodes()
print(f'Number of edges (gc): {e} ({gc.number_of_edges() / e:.0%})')
print(f'Number of edges (gc): {n} ({gc.number_of_nodes() / n:.0%})')
print(f'Density: {nx.density(g):.1e}')
print(f"Mean distance: {weighted_median(np.loadtxt('temp/output.txt')):.1f}")
print(f"Diameter: {int(np.loadtxt('temp/output.txt')[-1,0])}")

Number of edges (gc): 1301887 (100%)
Number of edges (gc): 545196 (100%)
Density: 8.8e-06
Mean distance: 4.8
Diameter: 11
